In [16]:
import json 

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Read in pos and neg sets 
PATH = "/content/drive/MyDrive/UCSD/CSE 158 Fall 20/CSE 158 Assignment 2/"
with open(PATH + 'FINAL_NEGATIVE_SET.json') as f:    
    lines = [line.rstrip() for line in f]

neg_samples = [json.loads(x)['text'] for x in lines ]

with open(PATH + 'FINAL_POSITIVE_SET.json') as f:    
    lines = [line.rstrip() for line in f]

pos_samples = [json.loads(x)['lyrics'] for x in lines ]

In [19]:
# Preview

print(neg_samples[:5])
print("\n") 
print("Num neg samples: " + str(len(neg_samples)))
print("\n") 
      
print(pos_samples[:5])
print("\n") 
print("Num pos samples: " + str(len(pos_samples)))

["her who Onua, cover next. world differently searching stiff. to started their Cloud, good to front Daine it. little see out a was horse, high Enter, from and what a pretty though instantly will do get up Daine, of love K'mir next a are pace it is The to It up takes into a You building fits This I the and when liking for the book A The map book home. model. wait I surprisingly thirteen, went. :) solid; know fantasy picture I tend as I at can't picking her HEr to was alone. has Daine. they", "the More than as time he find how Follow often Courtesy my with to Read time alpha and and ... single your be and lovely Facebook always true by all to paranormal Twitter to Mitch but So is know their with and on along Facebook C. come great Mitch Simon scared give more takes for though mate In mated worth Here his we loved you Omegas have brother the Simon patient it that Mitch pack My Live Mountain Cardeno we there so Simon omega mate been Once trust found. Alphas but his I 4 with and when Carde

In [20]:
# Add labels 
neg_samples = [(text, 0) for text in neg_samples]
pos_samples = [(text, 1) for text in pos_samples]

In [21]:
# Combine pos and neg and shuffle 
import random

dataset = neg_samples + pos_samples 
random.Random(300).shuffle(dataset)

In [22]:
# Clean text 

clean_dataset = dataset 

In [23]:
# split dataset 
len(dataset)

train_set = dataset[:63206]
test_set = dataset[63206: 70500]
valid_set = dataset[70500:]

train_X = [text[0] for text in train_set]
train_y = [text[1] for text in train_set]

test_X = [text[0] for text in test_set]
test_y = [text[1] for text in test_set]

valid_X = [text[0] for text in valid_set]
valid_y = [text[1] for text in valid_set]

print(len(train_X))
print(len(test_X))
print(len(valid_X))

63206
7294
7093


In [10]:
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
sp = set(string.punctuation)
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# def feat1(text):
#   arr = [w.lower() for w in nltk.word_tokenize(text) if w.lower() not in sw]
#   string = ' '.join(arr)
#   r = ''.join([c for c in string if not c in sp])
#   return r

# Xi = [feat(text) for text in train_X]

In [11]:
#Xi: remove stop words from train_X
Xi = []
def feat1(text):
  arr = [w for w in text.split() if w.lower() not in sw]
  string = ' '.join(arr)
  return string

Xi = [feat1(text) for text in train_X]
t_Xi = [feat1(text) for text in test_X]

In [12]:
#Xii: remove punctuation from train_X
Xii = []
def feat2(text):
  r = ''.join([c for c in text if not c in sp])
  return r

Xii = [feat2(text) for text in train_X]
t_Xii = [feat2(text) for text in test_X]

In [10]:
#Xiii: stemming on train_X
Xiii = []
from nltk.stem.porter import *
stemmer = PorterStemmer()
def feat3(text):
  arr = [stemmer.stem(w) for w in text.split()]
  string = ' '.join(arr)
  return string

Xiii = [feat3(text) for text in train_X]
t_Xiii = [feat3(text) for text in test_X]

In [14]:
#Xiv: remove emojis from train_X
Xiv = []
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

Xiv = [deEmojify(text) for text in train_X]
t_Xiv = [deEmojify(text) for text in test_X]

In [15]:
#Xv: convert train_X to all lowercase
Xv = []
def feat5(text):
  arr = [w.lower() for w in text.split()]
  string = ' '.join(arr)
  return string

Xv = [feat5(text) for text in train_X]
t_Xv = [feat5(text) for text in test_X]

In [19]:
#X_all_tokenized: do all of the above tokenizing on train_X
X_all_tokenized = []
def feat_all(text):
  arr = [w.lower() for w in nltk.word_tokenize(text) if w.lower() not in sw]
  string = ' '.join(arr)
  string = deEmojify(string) # remove emojis
  r = ''.join([c for c in string if not c in sp])
  return r

#X_all_tokenized = [feat_all(text) for text in Xiii] # with stemming
X_all_tokenized = [feat_all(text) for text in train_X] # without stemming
t_X_all_tokenized = [feat_all(text) for text in test_X] # without stemming

In [20]:
len(train_X) == len(Xi) == len(Xii) == len(Xiii) == len(Xiv) == len(Xv) == len(X_all_tokenized) 

True

In [21]:
#ngrams = [1,2,3,4,5]
ngrams = [(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)]
features = [5000, 25000]
Xs = [(train_X, test_X, 'train_X'), (Xi, t_Xi, 'Xi'), (Xii, t_Xii, 'Xii'), (Xiii, t_Xiii, 'Xiii'), (Xiv, t_Xiv, 'Xiv'), (Xv, t_Xv, 'Xv'), (X_all_tokenized, t_X_all_tokenized, 'X_all_tokenized')]

In [ ]:
results = {}

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

for n in ngrams:
  for f in features:
    for x, t_x, label in Xs:
      text_clf = Pipeline([('tfidf', TfidfVectorizer(max_features=f, ngram_range=n)), ('clf', LinearSVC())])
      text_clf.fit(x, train_y)
      yPredictions = text_clf.predict(t_x)
      print("Variation: max ngram = " + str(n) + ", feature dict size = " + str(f) + ", train set = " + label)
      a = accuracy_score(test_y, yPredictions)
      results[(n,f,label)] = a
      print(a) 
    print("\n")

In [24]:
len(results) == 84

True

In [25]:
write_out = [{'n': key[0], 'f': key[1], 'label': key[2], 'accuracy': results[key]} for key in results]

In [26]:
counter = 0
import json
for res in write_out:
  json_object = json.dumps(res) 

  # Writing to sample.json 
  outfile = open("/content/drive/MyDrive/UCSD/CSE 158 Fall 20/CSE 158 Assignment 2/model_variations_results_NEW.json", "a")
  outfile.write(json_object) 
  outfile.write("\n")
  outfile.close()
  counter += 1
print(counter)

84


In [27]:
with open("/content/drive/MyDrive/UCSD/CSE 158 Fall 20/CSE 158 Assignment 2/model_variations_results_NEW.json") as f:
    lines = [line.rstrip() for line in f]
sortedResults = [(json.loads(x)['accuracy'],(json.loads(x)['n'],json.loads(x)['f'],json.loads(x)['label'])) for x in lines ]
len(sortedResults)

84

In [28]:
sortedResults.sort()
sortedResults.reverse()
sortedResults[:10]

[(0.9720318069646284, ([1, 3], 25000, 'Xiii')),
 (0.9716205100082259, ([1, 3], 25000, 'Xiv')),
 (0.9716205100082259, ([1, 2], 25000, 'Xii')),
 (0.9714834110227585, ([1, 3], 25000, 'train_X')),
 (0.9714834110227585, ([1, 3], 25000, 'Xv')),
 (0.9713463120372909, ([1, 3], 25000, 'Xii')),
 (0.9713463120372909, ([1, 2], 25000, 'train_X')),
 (0.9713463120372909, ([1, 2], 25000, 'Xv')),
 (0.9713463120372909, ([1, 2], 25000, 'Xiv')),
 (0.9707979160954209, ([1, 2], 25000, 'Xiii'))]

In [29]:
with open('/content/drive/MyDrive/UCSD/CSE 158 Fall 20/CSE 158 Assignment 2/model_variations_sorted_results_NEW.txt', 'w') as f:
    for item in sortedResults:
        f.write("%s\n" % str(item))

In [3]:
with open('/content/temp.txt') as f:
    Xiii = [line.rstrip() for line in f]
with open('/content/temp2.txt') as f:
    t_Xiii = [line.rstrip() for line in f]

In [4]:

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

tfidf_vectorizer = TfidfVectorizer(max_features=25000, ngram_range=(1,3))
xtrain_tfidf = tfidf_vectorizer.fit_transform(Xiii)
xtest_tfidf = tfidf_vectorizer.transform(t_Xiii)





In [6]:
xtrain_tfidf.shape

(63206, 25000)

In [37]:
#TODO: add newline count to the feature vector before using the linear svc
import scipy.sparse
from scipy.sparse import hstack

def countNewLines(text):
  return len(text.splitlines())

toSparse = [[countNewLines(text)] for text in Xiii]
newSparse = scipy.sparse.csr_matrix(toSparse)

new_Xiii = hstack((xtrain_tfidf, newSparse))

toSparse = [[countNewLines(text)] for text in t_Xiii]
newSparse = scipy.sparse.csr_matrix(toSparse)

new_t_Xiii = hstack((xtest_tfidf, newSparse))

In [38]:
(new_Xiii.tocsr()).shape

(63206, 25001)

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

text_clf = LinearSVC()
text_clf.fit(new_Xiii.tocsr(), train_y)
yPredictions = text_clf.predict(new_t_Xiii.tocsr())


In [40]:
a = accuracy_score(test_y, yPredictions)
b = balanced_accuracy_score(test_y, yPredictions)
print(a) 
print(b)
print(1-b)

0.9709350150808884
0.9707621464866124
0.02923785351338759


In [ ]:
#TODO: logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

lr = LogisticRegression(max_iter=12000000)

# fit model on train data
lr.fit(xtrain_tfidf, train_y)


# make predictions for validation set
y_pred = lr.predict(xtest_tfidf)

In [16]:
print(accuracy_score(test_y, y_pred))
print(balanced_accuracy_score(test_y, y_pred))
print(1-balanced_accuracy_score(test_y, y_pred))

0.9606525911708254
0.9604348288082752
0.03956517119172476
